In [ ]:
# 01_ETL_and_Exploration.ipynb

# 📦 Import required modules
import pandas as pd
import sqlite3
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.etl_loader import run_etl

# 📁 Define paths
base_path = Path("../")
data_path = base_path / "data"
db_path = base_path / "db" / "healthcare_fraud.db"

# ✅ Step 1: Run ETL to generate merged CSV + SQLite DB
print("Running ETL pipeline...")
run_etl()

# ✅ Step 2: Load merged CSV for inspection
df = pd.read_csv(data_path / "processed_train.csv")
print("🔍 Sample of processed merged dataset:")
display(df.head())

# ✅ Step 3: Connect to SQLite DB for demonstration
conn = sqlite3.connect(db_path)
print("✅ Connected to SQLite DB")

# ✅ Step 4: Show all tables in SQLite (for viva/presentation)
print("📄 Tables in DB:")
tables_df = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
display(tables_df)

# ✅ Step 5: Sample SQL Query – Count inpatient claims
query_1 = """
SELECT COUNT(*) as total_inpatient_claims
FROM train_inpatient
"""
inpatient_count = pd.read_sql(query_1, conn)
print("📊 Total inpatient claims:")
display(inpatient_count)

# ✅ Step 6: Sample SQL Query – Avg Deductible Paid
query_2 = """
SELECT AVG(DeductibleAmtPaid) as avg_deductible_paid
FROM train_outpatient
"""
avg_deductible = pd.read_sql(query_2, conn)
print("📊 Average deductible amount paid (Outpatient):")
display(avg_deductible)

# ✅ Step 7: Sample SQL Query – Avg Chronic Conditions by Gender
query_3 = """
SELECT Gender, 
       AVG(ChronicCond_Alzheimer) as avg_alzheimers,
       AVG(ChronicCond_Heartfailure) as avg_heartfailure
FROM train_beneficiary
GROUP BY Gender
"""
chronic_by_gender = pd.read_sql(query_3, conn)
print("📊 Chronic conditions by gender:")
display(chronic_by_gender)

# ✅ Step 8: Check for null values & structure
print("ℹ️ Data Info:")
display(df.info())

print("🔍 Missing Values:")
display(df.isna().sum().sort_values(ascending=False))

# ✅ Step 9: Class Distribution of Target Variable
print("📊 Fraud Label Distribution (1 = Fraud, 0 = Legitimate):")
display(df['PotentialFraud'].value_counts())

# ✅ Step 10: Visualize Class Distribution
sns.countplot(data=df, x='PotentialFraud')
plt.title("Class Distribution: Fraud vs Non-Fraud")
plt.xlabel("Fraud (1 = Yes)")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig("../outputs/class_distribution.png")
plt.show()

# ✅ Step 11: Close DB connection
conn.close()
print(" Notebook completed.")
